In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import PIL
import glob
import math
import os
import torchvision
import gym
import matplotlib.pyplot as plt
import copy
import time
import random
import mujoco
from scipy import stats

In [2]:
#创建actor网络
class actor_net(nn.Module):
    
    def __init__(self,N_S,N_A):
        super().__init__()
        self.linear1=nn.Linear(N_S,64)
        self.linear2=nn.Linear(64,128)
        self.linear3=nn.Linear(128,64)
        self.mean=nn.Linear(64,N_A)
        self.vair=nn.Linear(64,N_A)
    
    def forward(self,inputs):
        x=self.linear1(inputs)
        x=torch.relu(x)
        x=self.linear2(x)
        x=torch.relu(x)
        x=self.linear3(x)
        x=torch.relu(x)
        mean=torch.tanh(self.mean(x))
        vair=torch.exp(torch.tanh(self.vair(x))-2)
        return mean,vair
    
    def choose_data(self,s):
        mu,va=self.forward(s)
        Pi=torch.distributions.Normal(mu,va)
        return Pi.sample().numpy()
#创建critic网路
class critic_net(nn.Module):
    
    def __init__(self,N_S):
        super().__init__()
        self.linear1=nn.Linear(N_S,64)
        self.linear2=nn.Linear(64,128)
        self.linear3=nn.Linear(128,64)
        self.out=nn.Linear(64,1)
    
    def forward(self,inputs):
        x=torch.relu(self.linear1(inputs))
        x=torch.relu(self.linear2(x))
        x=torch.relu(self.linear3(x))
        out=self.out(x)
        return out

In [3]:
actor=actor_net(27,8)
critic=critic_net(27)
MSE_loss=torch.nn.MSELoss()

In [4]:
actor_opt=torch.optim.Adam(actor.parameters(),lr=0.0001)
critic_opt=torch.optim.Adam(critic.parameters(),lr=0.0001)

In [160]:
#学习优化的方向：states归一化怎么实现，计算优势使用GAE，Advantage归一化？（这个不一定有）
%matplotlib qt5
epoches=2000 #设置训练epoches
batch_size=2500#设置batch_size
buffer_length=25000#replay_buffer中一次存这些个数据，数据格式：[array(states),array(action),int(reward),int(mask)]
discount=0.99#折扣回报率
env=gym.make('Ant-v4',healthy_reward=0,ctrl_cost_weight=0)#创建环境
observation,info=env.reset()#初始化环境
# remember_reward=[]#用来储存每一个episode获得的分数总和
count_reward_plot=0#用来计算一个episode获得的总分
# plt.figure('Ant',figsize=(15,7))
for iter in range(epoches):
    replay_buffer=[]#replay_buffer储存容器
    #收集数据到memory:
    for collect in range(buffer_length):#建立获取经验的循环
        action=actor.choose_data(torch.from_numpy(observation).type(torch.FloatTensor))
        #调用actor_net类定义的choose_data函数来获取action
        observation_, reward, truncated, terminated, info=env.step(list(action))#执行antion
        count_reward_plot+=reward#计算episode的reward
        if truncated==0 and terminated==0:#判断这一局是否可以继续
            mask=0#如果不停止，mask设为0
        elif truncated==1:#如果ant失败了，mask设为1
            mask=1
#             plt.clf()
#             plt.plot(remember_reward,'r')
#             plt.show()
#             plt.pause(0.000001)
            remember_reward.append(count_reward_plot)#将本episode获得的分数储存起来
            count_reward_plot=0#变量重定为0
            observation_,info=env.reset()#环境初始化
        elif terminated==1:#如果本次episode步数达到1000，将mask设为2
            mask=2
#             plt.clf()
#             plt.plot(remember_reward,'r')
#             plt.show()
#             plt.pause(0.000001)
            remember_reward.append(count_reward_plot)
            count_reward_plot=0
            observation_,info=env.reset()
        replay_buffer.append([observation,action,reward,mask])#在replay_buffer中存储数据
        observation=observation_
    #计算advantage，存到count_advantage：
    count_TD_target=[]#用来储存每一个数据TD_target的容器
    for i in range(len(replay_buffer)):#建立遍历replay_buffer每一个数据的循环，用来计算每一个环境的TD_target
        if replay_buffer[i][3]==2:#如果当前这个数据的mask为2（代表本数据是到1000自动结束的），则不计算该数据的TD_target
            continue
        if i==len(replay_buffer)-1:#如果当前数据是replay_buffer的最后一个，则不计算该数据的TD_target，因为没法计算
            continue
        count_score=0#该变量用来计算折扣回报加和
        #以下用来循环计算某步的return(TD_traget)
        for j in range(1000):#因为1000步是一个episode最大步数，所以建立1000的循环
            if (i+j)==len(replay_buffer)-1:#如果当前数据是replay_buffer的最后一个数据，则加critic预测出来的数乘折扣率
                if replay_buffer[i+j][3]==1:
                    count_score+=replay_buffer[i+j][2]*(discount**j)
                else:
                    count_score+=critic(torch.from_numpy(replay_buffer[i+j][0]).type(torch.FloatTensor)).item() \
                    *(discount**j)
                break
            if replay_buffer[i+j][3]==1:#如果当前数据的mask=1，代表本次数据ant失败了，则加上这次数据的折扣回报就跳出循环
                count_score+=replay_buffer[i+j][2]*(discount**j)
                break
            elif replay_buffer[i+j][3]==0:#如果当前数据的mask=0，代表ant正常允许，加上本次数据的折扣回报后，继续循环
                count_score+=replay_buffer[i+j][2]*(discount**j)
            elif replay_buffer[i+j][3]==2:#如果当前数据的mask=2，代表遇到了1000步停止的，则加critic预测出来的数乘折扣率
                count_score+=critic(torch.from_numpy(replay_buffer[i+j][0]).type(torch.FloatTensor)).item() \
                *(discount**j)
                break
        count_TD_target.append([count_score])
        #将replay_buffer中每一个数据对应的TD_target保存起来（除了mask=2的，之后要舍弃掉），注意格式，要与其他数据对应
    del replay_buffer[-1]#之前没有计算replay_buffer最后一个数据的TD_target，所以要删掉replay_buffer中的最后一个数据
    replay_buffer=[item for item in replay_buffer if item[3]!=2]#将mask=2的数据从replay_buffer中剔除
    #因为达到1000步自动停止的数据（mask=2），会影响运算，所以需要剔除这些数据，让replay_buffer和count_TD_target每个元素对应
    #计算旧策略的log_prob：
    states=torch.tensor([item[0] for item in replay_buffer]).type(torch.FloatTensor)
    #获取replay_buffer中每个数据的states，单独成一个列表，并转为tensor
    actions_=torch.tensor([item[1] for item in replay_buffer]).type(torch.FloatTensor)
    #获取replay_buffer中每个数据的actions，单独成一个列表，并转为tensor
    old_mu,old_std=actor(states)#用actor网络运算获得每个states对应的旧策略均值和旧策略方差
    pi=torch.distributions.Normal(old_mu,old_std)#获得旧策略的正态分布，每个states对应8个分布
    old_log_prob=pi.log_prob(actions_).sum(1,keepdim=True)
    #.log_prob()函数可以获得某分布取值对应的概率值的log值，sum(1)是将每个states对应的8个分布的log_prob加起来，
    #因为log(a*b)=loga+logb，keepdim=True是保持原有的数据格式，即为保持pi原有的数据格式,例如：
    #[[1,2,3],[3,4,5]].sum(1,keepdim=True)->[[6],[12]],如果keepdim=False,则输出为[6,12]
    #sum(0),sum(1),sum()三者输出不同，sum()将所有的数求合，得出最终一个数，sum(0)和sum(1)则求不同维度的和
    #以下开始训练：
    for batch_time in range(len(replay_buffer)//batch_size):#建立一个epoch的训练，更新次数与batch_size和buffer长度有关
        start_num=batch_size*batch_time#此次batch从第几个数据开始训练
        end_num=batch_size*batch_time+batch_size#此次batch结束于哪个数据
        states_batch=states[start_num:end_num]#获取在此batch区域内的环境states
        action_batch=actions_[start_num:end_num]#获取在此batch区域内的动作action
        TD_target_batch=torch.tensor(count_TD_target[start_num:end_num]).type(torch.FloatTensor)
        #获取在此batch区域内的TD_target，并转化为tensor格式
        batch_value=critic(states_batch)#使用当前critic计算此batch各个states的分数
        critic_loss=MSE_loss(batch_value,TD_target_batch)#使用MSE_loss，计算critic的损失，TD算法
        critic_opt.zero_grad()
        critic_loss.backward()
        critic_opt.step()
        new_mu,new_div=actor(states_batch)#使用当前的actor计算新策略均值和新策略方差
        pi=torch.distributions.Normal(new_mu,new_div)#获得新策略的正态分布，每个states对应8个分布
        new_log_pron=pi.log_prob(action_batch).sum(1,keepdim=True)#获得新分布动作采样概率乘积的log值，注意保持维度
        ratio=torch.exp(new_log_pron-(old_log_prob[start_num:end_num]).detach())
        #计算此batch中，新分布动作采样概率与旧分布动作采样概率的比值，使用相减的原因：log(a/b)=loga-logb，再用exp计算去掉log
        actor_loss=-torch.min(ratio*(TD_target_batch-batch_value.detach()),torch.clip(ratio,0.8,1.2)* \
                              (TD_target_batch-batch_value.detach())).mean()
        #使用PPO2算法计算actor的损失，最前面加一个-号，因为使用Adam优化器是梯度下降，而PPO算法要求的应该是梯度上升，加一个-号
        #就可以使用梯度下降了
        actor_opt.zero_grad()
        actor_loss.backward()
        actor_opt.step()

KeyboardInterrupt: 

In [2]:
ooo=[[1],[3],[4]]

In [3]:
del ooo[-1]

In [4]:
ooo

[[1], [3]]

In [178]:
plt.figure(figsize=(20,10))
plt.plot(remember_reward)

In [7]:
%matplotlib qt5
env=gym.make("Ant-v4",healthy_reward=0,ctrl_cost_weight=0,render_mode='rgb_array')
observation,info=env.reset()
times=0
plt.figure('Ant',figsize=(6,6))
while(1):
    plt.clf()
    plt.imshow(env.render())
    plt.pause(0.00000001)
    with torch.no_grad():
        actionxxx=actor.choose_data(torch.from_numpy(observation).type(torch.FloatTensor))
    observation, reward, truncated, terminated, info=env.step(actionxxx)
    times+=1
    if terminated:
        print('end by 1000,times=',times)
        print(info)
        break
    elif truncated:
        print('end by fail,times=',times)
        print(info)
        break
env.close()
print('end')

KeyboardInterrupt: 

In [49]:
torch.save(actor.state_dict(),'./MuJoCo_Antv4_PPO2_actor_state_dict.pth')
torch.save(critic.state_dict(),'./MuJoCo_Antv4_PPO2_critic_state_dict.pth')

In [6]:
actor.load_state_dict(torch.load('./MuJoCo_Antv4_PPO2_actor_state_dict.pth'))
critic.load_state_dict(torch.load('./MuJoCo_Antv4_PPO2_critic_state_dict.pth'))

<All keys matched successfully>